# 自定义层

深度学习成功背后的一个因素是，可以用创造性的方式组合广泛的层，从而设计出适用于各种任务的结构。例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态编程的层。早晚有一天，你会遇到或要自己发明一个在深度学习框架中还不存在的层。在这些情况下，你必须构建自定义层。在本节中，我们将向你展示如何操作。

## 不带参数的层

首先，我们(**构造一个没有任何参数的自定义层**)。如果你还记得我们在 :numref:`sec_model_construction` 对块的介绍，这应该看起来很眼熟。下面的`CenteredLayer`类要从其输入中减去均值。要构建它，我们只需继承基础层类并实现正向传播功能。


In [1]:
import tensorflow as tf

class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

让我们通过向其提供一些数据来验证该层是否按预期工作。


In [2]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

现在，我们可以[**将层作为组件合并到构建更复杂的模型中**]。


In [3]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

作为额外的健全性检查，我们可以向网络发送随机数据后，检查均值是否为0。由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。


In [4]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=2.3283064e-10>

## [**带参数的层**]

既然我们知道了如何定义简单的层，那么让我们继续定义具有参数的层，这些参数可以通过训练进行调整。我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。比如管理访问、初始化、共享、保存和加载模型参数。这样做的好处之一是，我们不需要为每个自定义层编写自定义序列化程序。

现在，让我们实现自定义版本的全连接层。回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。在此实现中，我们使用ReLU作为激活函数。该层需要输入参数：`in_units`和`units`，分别表示输入和输出的数量。


In [5]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(
            name='weight', shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(name='bias', shape=[self.units],
                                    initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

接下来，我们实例化`MyDense`类并访问其模型参数。


In [6]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.07033903, -0.09516857, -0.04932721],
        [ 0.0816838 , -0.03303741,  0.0049427 ],
        [ 0.00819202,  0.00020935, -0.05778468],
        [-0.03122104,  0.00289387,  0.01236265],
        [-0.05175416,  0.11570921, -0.06432124]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

我们可以[**使用自定义层直接执行正向传播计算**]。


In [7]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.02560703, 0.03174396, 0.        ],
       [0.05324733, 0.        , 0.        ]], dtype=float32)>

我们还可以(**使用自定义层构建模型**)。我们可以像使用内置的全连接层一样使用自定义层。


In [8]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.02445628],
       [0.04983368]], dtype=float32)>

## 小结

* 我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与库中的任何现有层不同。
* 在自定义层定义完成后，就可以在任意环境和网络结构中调用该自定义层。
* 层可以有局部参数，这些参数可以通过内置函数创建。

## 练习

1. 设计一个接受输入并计算张量降维的层，它返回$y_k = \sum_{i, j} W_{ijk} x_i x_j$。
1. 设计一个返回输入数据的傅立叶系数前半部分的层。


[Discussions](https://discuss.d2l.ai/t/1836)
